# 1. Load Environment & Huggingface, W&B login

In [1]:
import wandb
import os
os.environ["WANDB_PROJECT"]="QLoRA_Instruction_finetune_02"

wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: aeolian83. Use `wandb login --relogin` to force relogin


True

In [2]:
from datasets import load_dataset
from huggingface_hub import login
from dotenv import load_dotenv

load_dotenv()


login(token= os.environ["HF_TOKEN"])

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/aeolian83/.cache/huggingface/token
Login successful


# 2. Dataset Load

- 이 시스템에서는 작업을 외장하드에서 하고 있기 때문에 캐쉬폴더를 별도로 지정(In this system, tasks are being performed on an external hard drive, so a separate cache folder is specified.)

In [3]:
ko_instruction_01 = load_dataset("squarelike/OpenOrca-gugugo-ko", cache_dir="/mnt/t7/.cache/huggingface/datasets")

In [4]:
# ko_instruction_01 = load_dataset("squarelike/OpenOrca-gugugo-ko", cache_dir="/mnt/t7/.cache/huggingface/datasets")

In [5]:
ko_instruction_01 = ko_instruction_01.shuffle(seed=2160)

In [6]:
ko_instruction_01

DatasetDict({
    train: Dataset({
        features: ['question', 'system_prompt', 'response', 'id'],
        num_rows: 2240294
    })
})

In [7]:
ko_instruction_01 = ko_instruction_01['train'].train_test_split(test_size=0.95)

In [8]:
ko_instruction_01

DatasetDict({
    train: Dataset({
        features: ['question', 'system_prompt', 'response', 'id'],
        num_rows: 112014
    })
    test: Dataset({
        features: ['question', 'system_prompt', 'response', 'id'],
        num_rows: 2128280
    })
})

In [9]:
ko_instruction_01['train'][1500]['question']

'* 박테리아 또는 바이러스 감염: 바이러스 및 곰팡이 수막염 및 결핵과 같은 감염은 척추에 영향을 줄 수 있습니다.\n\n단어 사이에 공백을 제거하세요.'

In [10]:
ko_instruction_01['train'][1500]['system_prompt']

'당신은 인공지능 비서입니다. 당신은 임무를 부여받을 것입니다. 당신은 상세하고 긴 답변을 작성해야 합니다.'

In [11]:
ko_instruction_01['train'][1500]['response']

'박테리아 또는 바이러스에 의한 감염은 척추 문제를 포함한 다양한 건강 문제를 유발할 수 있습니다. 예를 들어, 바이러스 및 곰팡이 수막염과 결핵은 척추에 영향을 줄 수 있는 감염 중 일부입니다. 수막염은 뇌와 척수를 감싸고 있는 보호막이 염증으로 인해 부어오르는 것으로, 바이러스 또는 곰팡이 감염에 의해 발생할 수 있습니다. 감염이 척추로 퍼지면 척추 수막염을 유발하여 심한 통증, 뻣뻣함, 부기를 유발할 수 있습니다. 결핵은 척추에 영향을 줄 수 있는 또 다른 감염입니다. 주로 폐에 영향을 미치지만 척추를 포함한 신체의 다른 부위로도 퍼질 수 있는 박테리아 감염입니다. 결핵이 척추에 영향을 받으면 척추 결핵 또는 포트병이라고 합니다. 이 질환은 심한 통증, 뻣뻣함, 척추 기형과 함께 신경학적 문제를 유발할 수 있으며, 치료하지 않으면 신경학적 문제로 이어질 수 있습니다.\n\n결론적으로 박테리아 또는 바이러스에 의한 감염은 척추에 심각한 결과를 초래할 수 있습니다. 척추에 심한 통증, 뻣뻣함, 부기 등 척추 감염 증상이 나타나면 의료진의 진료를 받는 것이 중요합니다. 조기 진단과 치료는 추가 합병증을 예방하고 회복 가능성을 높이는 데 도움이 될 수 있습니다.'

# 3. Model Load

- QLoRA로 finetune을 하기위해 일반 7B 모델을 BitsAndBytes를 통해 양자화(4bit quantization)하여 로드, 훈련속도 향상을 위해 데이터 타입을 fp32가 아니라 fp16으로 로드(차후에는 bf16으로 로드해서 훈련예정)
- To finetune with QLoRA, a standard 7B model is quantized (4-bit quantization) through BitsAndBytes for loading, and to improve training speed, the data type is loaded as fp16 instead of fp32 (with plans to load and train with bf16 in the future).


```python

quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        llm_int8_threshold=6.0,
        llm_int8_has_fp16_weight=False,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
    ),
```

### *Reference

- https://github.com/huggingface/peft/blob/main/examples/fp4_finetuning/finetune_fp4_opt_bnb_peft.py
- https://colab.research.google.com/drive/19AFEOrCI6-bc7h9RTso_NndRwXJRaJ25?usp=sharing

In [12]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, PeftModel, get_peft_model
from trl import SFTTrainer

In [13]:
checkpoint_dir = "./checkpoint"
model_id = "beomi/llama-2-ko-7b"
device_map = {"": 0}

In [14]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        llm_int8_threshold=6.0,
        llm_int8_has_fp16_weight=False,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
    ),
    device_map=device_map,
    torch_dtype=torch.bfloat16,
    cache_dir="/mnt/t7/.cache/huggingface/models",
)
tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir="/mnt/t7/.cache/huggingface/models")

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

## (1) Check Trainable Parameter

- 아래 코드대로 모델을 로드하면 훈련하는 파라미터수를 확인할 수 있다. 

In [15]:
# from peft import prepare_model_for_kbit_training

# model.gradient_checkpointing_enable()
# model = prepare_model_for_kbit_training(model)

In [16]:
# def print_trainable_parameters(model):
#     """
#     Print the number of trainable parameters in the model.
#     """
#     trainable_params = 0
#     all_param = 0
#     for _, param in model.named_parameters():
#         all_param += param.numel()
#         if param.requires_grad:
#             trainable_params += param.numel()
#     print(
#         f"trainable params: {trainable_params} || all params: {all_param} || trainble%: {100 * trainable_params / all_param}"
#     )

### 1) PEFT훈련을 위한 CONFIG설정, 이 부분은 Trainable Parameter를 확인하지 않아도 QLoRA를 위해서는 무조건 해야 한다.("For PEFT training configuration, this step must be done unconditionally for QLoRA, even without checking for Trainable Parameters.")

In [17]:
# peft_config = LoraConfig(
#     r=64,
#     lora_alpha=16,
#     lora_dropout=0.1,
#     inference_mode=False,
#     task_type="CAUSAL_LM",
# )

peft_config = LoraConfig(
    r=64,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj", "out_proj", "fc1", "fc2"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

In [18]:
# model = get_peft_model(model, peft_config)
# print_trainable_parameters(model)


### 2) 훈련을 위한 모델 설정(Model configuration for training)

In [19]:
model.config.use_cache = False # silence the warnings. Please re-enable for inference!
model.config.pretraining_tp = 1

### 3) loss값이 치솟는걸 방지하기 위한 토크나이저 설정(Tokenizer configuration to prevent surging loss values)

In [20]:
# tokenizer.pad_token = tokenizer.eos_token

# # 학습 진행 중 loss가 치솟다가 0.0으로 떨어지는 문제 해결을 위해 사용
tokenizer.padding_side = "right"

### 4) Prompt 설정(Prompt configuration)

- Prompt는 아래와 같음(The prompt is as follows.)
> \#\#\# System: content  
> \#\#\# Human: content  
> \#\#\# Assistant: content  

- Prompt에 맞춰서 Dataset을 재가공(Reprocessing the Dataset according to the Prompt)

In [21]:
def format_instruction(sample):
    system_prompt = f"### System: {sample['system_prompt']}"
    input = f"### Human: {sample['question']}" if len(sample["question"]) > 0 else None
    output = f"### Assistant\n{sample['response']}"
    # join all the parts together
    prompt = "\n\n".join([i for i in [system_prompt, input, output] if i is not None])
    return prompt

# template dataset to add prompt to each sample
def template_dataset(sample):
    sample["text"] = f"{format_instruction(sample)}{tokenizer.eos_token}"
    return sample

In [22]:
train_dataset = ko_instruction_01['train'].map(template_dataset, remove_columns=list(ko_instruction_01['train'].features), num_proc=10)

Map (num_proc=10):   0%|          | 0/112014 [00:00<?, ? examples/s]

In [23]:
# test_dataset = ko_instruction_01['test'].map(template_dataset, remove_columns=list(ko_instruction_01['test'].features), num_proc=10)

# 4. Configure Train arguments & Define Trainer

In [24]:
training_arguments = TrainingArguments(
    output_dir=checkpoint_dir,
    save_steps=200,
    save_total_limit=3, # 가장 최근 체크포인트 3개만 저장합니다.
    logging_steps=30,
    report_to="wandb",
    learning_rate=1e-5,
    max_grad_norm=0.3,
    num_train_epochs=3, # epochs 대신 max_steps을 기준으로 할 수 있습니다.
    warmup_ratio=0.01,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit", # paged_adamw_8bit 사용시 메모리를 더 절약할 수 있지만 loss가 0으로 떨어지는 문제가 있습니다.
    group_by_length=True,
    fp16 = False, # 코랩 무료버전에서 실행 시 "True"를 사용하세요
    bf16 = True, # 코랩 무료버전에서 실행 시 "False"를 사용하세요
    lr_scheduler_type="cosine",
)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=1024,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False,
)

Map:   0%|          | 0/112014 [00:00<?, ? examples/s]

/home/aeolian83/anaconda3/envs/llm_for_p311/lib/python3.11/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [25]:
trainer.train()


  0%|          | 0/336042 [00:00<?, ?it/s]

{'loss': 2.9502, 'grad_norm': 0.41015625, 'learning_rate': 8.925914906277894e-08, 'epoch': 0.0}
{'loss': 3.667, 'grad_norm': 0.314453125, 'learning_rate': 1.7851829812555787e-07, 'epoch': 0.0}
{'loss': 3.4188, 'grad_norm': 0.462890625, 'learning_rate': 2.677774471883368e-07, 'epoch': 0.0}
{'loss': 3.3953, 'grad_norm': 0.353515625, 'learning_rate': 3.5703659625111574e-07, 'epoch': 0.0}
{'loss': 3.9255, 'grad_norm': 0.66796875, 'learning_rate': 4.462957453138947e-07, 'epoch': 0.0}
{'loss': 3.3231, 'grad_norm': 0.443359375, 'learning_rate': 5.355548943766736e-07, 'epoch': 0.0}
{'loss': 3.8258, 'grad_norm': 0.234375, 'learning_rate': 6.248140434394526e-07, 'epoch': 0.0}
{'loss': 3.205, 'grad_norm': 0.3828125, 'learning_rate': 7.140731925022315e-07, 'epoch': 0.0}
{'loss': 3.4737, 'grad_norm': 0.2890625, 'learning_rate': 8.033323415650105e-07, 'epoch': 0.0}
{'loss': 3.8961, 'grad_norm': 0.921875, 'learning_rate': 8.925914906277894e-07, 'epoch': 0.0}
{'loss': 3.112, 'grad_norm': 0.53125, 'lea

/home/aeolian83/anaconda3/envs/llm_for_p311/lib/python3.11/site-packages/peft/utils/other.py:588: UserWarning: Unable to fetch remote file due to the following error 500 Server Error: Internal Server Error for url: https://huggingface.co/beomi/llama-2-ko-7b/resolve/main/config.json (Request ID: Root=1-6627f210-49153eba2f6a903454f23259;8392fb95-dc60-4f42-b487-4b741e9110a3)

Internal Error - We're working hard to fix this as soon as possible! - silently ignoring the lookup for the file config.json in beomi/llama-2-ko-7b.
  warnings.warn(
/home/aeolian83/anaconda3/envs/llm_for_p311/lib/python3.11/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in beomi/llama-2-ko-7b - will assume that the vocabulary was not modified.
  warnings.warn(


{'loss': 1.575, 'grad_norm': 1.0390625, 'learning_rate': 2.637309564664906e-06, 'epoch': 1.98}
{'loss': 1.6007, 'grad_norm': 2.546875, 'learning_rate': 2.6360612934284358e-06, 'epoch': 1.98}
{'loss': 1.5998, 'grad_norm': 1.65625, 'learning_rate': 2.6348132119160487e-06, 'epoch': 1.98}
{'loss': 1.6536, 'grad_norm': 3.265625, 'learning_rate': 2.633565320227914e-06, 'epoch': 1.98}
{'loss': 1.7369, 'grad_norm': 1.9296875, 'learning_rate': 2.6323176184641875e-06, 'epoch': 1.98}
{'loss': 1.8001, 'grad_norm': 1.6328125, 'learning_rate': 2.631070106725001e-06, 'epoch': 1.98}
{'loss': 1.7005, 'grad_norm': 4.1875, 'learning_rate': 2.629822785110484e-06, 'epoch': 1.98}
{'loss': 1.6763, 'grad_norm': 1.3984375, 'learning_rate': 2.6285756537207397e-06, 'epoch': 1.98}
{'loss': 1.6574, 'grad_norm': 4.15625, 'learning_rate': 2.6273287126558565e-06, 'epoch': 1.98}
{'loss': 1.6792, 'grad_norm': 2.25, 'learning_rate': 2.6260819620159184e-06, 'epoch': 1.98}
{'loss': 1.7636, 'grad_norm': 1.1953125, 'learnin

/home/aeolian83/anaconda3/envs/llm_for_p311/lib/python3.11/site-packages/peft/utils/other.py:588: UserWarning: Unable to fetch remote file due to the following error (ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 8169f23a-5eea-405d-87e3-e5f8b925797d)') - silently ignoring the lookup for the file config.json in beomi/llama-2-ko-7b.
  warnings.warn(
/home/aeolian83/anaconda3/envs/llm_for_p311/lib/python3.11/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in beomi/llama-2-ko-7b - will assume that the vocabulary was not modified.
  warnings.warn(


...

/home/aeolian83/anaconda3/envs/llm_for_p311/lib/python3.11/site-packages/peft/utils/other.py:588: UserWarning: Unable to fetch remote file due to the following error (ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 0ea72f6a-3e36-4d68-a51d-db64236b814b)') - silently ignoring the lookup for the file config.json in beomi/llama-2-ko-7b.
  warnings.warn(
/home/aeolian83/anaconda3/envs/llm_for_p311/lib/python3.11/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in beomi/llama-2-ko-7b - will assume that the vocabulary was not modified.
  warnings.warn(


{'loss': 1.7471, 'grad_norm': 3.15625, 'learning_rate': 4.160549073706221e-11, 'epoch': 3.0}
{'loss': 1.5907, 'grad_norm': 2.40625, 'learning_rate': 3.6027600818777654e-11, 'epoch': 3.0}
{'loss': 1.7505, 'grad_norm': 3.578125, 'learning_rate': 3.085099608945985e-11, 'epoch': 3.0}
{'loss': 1.676, 'grad_norm': 3.6875, 'learning_rate': 2.607567696377711e-11, 'epoch': 3.0}
{'loss': 1.7416, 'grad_norm': 1.265625, 'learning_rate': 2.1701643825866593e-11, 'epoch': 3.0}
{'loss': 1.7243, 'grad_norm': 2.34375, 'learning_rate': 1.772889702655878e-11, 'epoch': 3.0}
{'loss': 1.7103, 'grad_norm': 1.921875, 'learning_rate': 1.4157436884487675e-11, 'epoch': 3.0}


/home/aeolian83/anaconda3/envs/llm_for_p311/lib/python3.11/site-packages/peft/utils/other.py:588: UserWarning: Unable to fetch remote file due to the following error (ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 1fd047f5-fa10-41b6-b607-777641e04347)') - silently ignoring the lookup for the file config.json in beomi/llama-2-ko-7b.
  warnings.warn(
/home/aeolian83/anaconda3/envs/llm_for_p311/lib/python3.11/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in beomi/llama-2-ko-7b - will assume that the vocabulary was not modified.
  warnings.warn(


{'loss': 1.6454, 'grad_norm': 3.046875, 'learning_rate': 1.0987263686645933e-11, 'epoch': 3.0}
{'loss': 1.8438, 'grad_norm': 2.765625, 'learning_rate': 8.218377686719514e-12, 'epoch': 3.0}
{'loss': 1.6027, 'grad_norm': 1.1171875, 'learning_rate': 5.850779107863247e-12, 'epoch': 3.0}
{'loss': 1.773, 'grad_norm': 1.9140625, 'learning_rate': 3.8844681399252686e-12, 'epoch': 3.0}
{'loss': 1.6631, 'grad_norm': 1.8515625, 'learning_rate': 2.319444940002136e-12, 'epoch': 3.0}
{'loss': 1.6263, 'grad_norm': 5.1875, 'learning_rate': 1.155709634104163e-12, 'epoch': 3.0}
{'loss': 1.6632, 'grad_norm': 2.484375, 'learning_rate': 3.932623160451954e-13, 'epoch': 3.0}
{'loss': 1.6147, 'grad_norm': 2.09375, 'learning_rate': 3.210304633238792e-14, 'epoch': 3.0}
{'train_runtime': 130266.4907, 'train_samples_per_second': 2.58, 'train_steps_per_second': 2.58, 'train_loss': 1.707353885097414, 'epoch': 3.0}


TrainOutput(global_step=336042, training_loss=1.707353885097414, metrics={'train_runtime': 130266.4907, 'train_samples_per_second': 2.58, 'train_steps_per_second': 2.58, 'train_loss': 1.707353885097414, 'epoch': 3.0})

### 허깅페이스에서 훈련 도중 러닝레이트가 수정 안되는 경우가 많다 한다. 
- 스케쥴러 변경을 시도해 보기로 한다. 

In [26]:
output_dir = "./results/experi_02"

In [27]:
trainer.model.save_pretrained(output_dir)

/home/aeolian83/anaconda3/envs/llm_for_p311/lib/python3.11/site-packages/peft/utils/other.py:588: UserWarning: Unable to fetch remote file due to the following error (ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 3c877566-c0c7-4b66-a1ec-258126076aba)') - silently ignoring the lookup for the file config.json in beomi/llama-2-ko-7b.
  warnings.warn(
/home/aeolian83/anaconda3/envs/llm_for_p311/lib/python3.11/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in beomi/llama-2-ko-7b - will assume that the vocabulary was not modified.
  warnings.warn(
